# Welcome to the 210Pb age model script!

### <div style="text-align: right"> Last modified by A.A. Lehrmann 25 June 2025 </div>


### The script below will extract radioisotope data from Canberra PDFs, run the age model (from the Wellner Lab Group excel model (Appleby, 2001; Boldt et al., 2013), and plot the age model

### Important instructions before you begin:

    1. NEVER edit raw data. Do not delete Canberra PDFs. Do not remove sediment weights from original lab notebook excel sheet.

    2. Make an /CORE_AgeModelOutput/ folder to put all of your script's outputs

    3. When copying folder paths, make sure to remove quotation marks

    4. Always add the extension .csv to your output files

## First, extract radioisotope data from Canberra PDFs
Run cell (press triangle that says run) and follow instructions.

In [ ]:
#import required libraries
import os
import pandas as pd
from PyPDF2 import PdfReader

In [ ]:
#Process pt_src files
def process_ptsrc_pdf(file_path, filename):
    try:
        reader = PdfReader(file_path)
        if len(reader.pages) < 3:
            print(f"PDF file '{filename}' has less than 3 pages. Skipping.")
            return None
        page = reader.pages[2]
        text = page.extract_text()
        lines = text.split('\n')
        for line in lines:
            if 'Pb-210' in line:
                ptsrc_pb210, PtSrc_Pb210error = line.split()[-2:]
                return {
                    'File': filename,
                    'Pb-210': float(ptsrc_pb210),
                    'Pb-210 error': float(PtSrc_Pb210error)
                }
        print(f"Pb-210 not found in '{filename}'. Skipping.")
        return None
    except Exception as e:
        print(f"Error processing PDF file '{filename}': {e}")
        return None


In [ ]:
#Process regular files
def process_regular_pdf(file_path, filename):
    pb210 = pb210error = Bi214 = Bi214error = Pb214 = Pb214error = None
    try:
        reader = PdfReader(file_path)
        if len(reader.pages) < 3:
            print(f"PDF file '{filename}' has less than 3 pages. Skipping.")
            return None
        page = reader.pages[2]
        text = page.extract_text()
        lines = text.split('\n')
        for line in lines:
            if 'Pb-210' in line:
                pb210, pb210error = line.split()[-2:]
            elif 'Bi-214' in line:
                Bi214, Bi214error = line.split()[-2:]
            elif 'Pb-214' in line:
                Pb214, Pb214error = line.split()[-2:]
        if pb210 is None or pb210error is None:
            print(f"Pb-210 not found in '{filename}'. Skipping.")
            return None
        if Bi214 is None or Bi214error is None:
            print(f"Bi-214 not found in '{filename}'. Skipping.")
            return None
        if Pb214 is None or Pb214error is None:
            print(f"Pb-214 not found in '{filename}'. Skipping.")
            return None
        return {
            'File': filename,
            'Pb-210': float(pb210),
            'Pb-210 error': float(pb210error),
            'Bi-214': float(Bi214),
            'Bi-214 error': float(Bi214error),
            'Pb-214': float(Pb214),
            'Pb-214 error': float(Pb214error)
        }
    except Exception as e:
        print(f"Error processing PDF file '{filename}': {e}")
        return None

In [ ]:
#define PDF processing
def process_pdf_file(file_path, filename):
    if filename.startswith("PtSrc_"):
        return process_ptsrc_pdf(file_path, filename)
    else:
        return process_regular_pdf(file_path, filename)

In [ ]:
#Extract PDF data from folder
def extract_pdf_data(folder_path):
    combined_data = []
    for filename in os.listdir(folder_path):
        # Check for PDF extension (case-insensitive)
        if filename.lower().endswith(".pdf"):
            file_path = os.path.join(folder_path, filename)
            data = process_pdf_file(file_path, filename)
            if data is not None:
                combined_data.append(data)
    return combined_data

In [ ]:
#Sort and process PDFs
def sort_pdf_data(combined_data, parse_numbers=False):
    combined_df = pd.DataFrame(combined_data)
    
    def extract_numeric_suffix(file_name):
        try:
            parts = file_name.split('_')[-1].split('.')[0]
            return int(parts)
        except ValueError:
            return float('nan')
    
    combined_df['File_order'] = combined_df['File'].apply(extract_numeric_suffix)
    combined_df = combined_df.sort_values(by='File_order').drop(columns=['File_order'])
    
    if parse_numbers:
        for col in ['Pb-210', 'Bi-214', 'Pb-214']:
            if col in combined_df.columns:
                combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')
    return combined_df

In [ ]:
#Export to csv
def extract_pdf_values(folder_path, output_csv_path, parse_numbers=False):
    combined_data = extract_pdf_data(folder_path)
    combined_df = sort_pdf_data(combined_data, parse_numbers)
    combined_df.to_csv(output_csv_path, index=False)

In [ ]:
#User inputs
folder_path = input("Enter the folder path of Canberra PDFs: ")
output_csv_path = input("Enter the output CSV file path (e.g. CORE_CanberraData_DATE.csv): ")

In [ ]:
#Execute functions
extract_pdf_values(folder_path, output_csv_path)

### Make note of which samples are missing data! This will be important when we plot!

# Open the output .csv file

Check to make sure all radioisotope data translated correctly. 

# Create two new columns
- ptsrc_pb210
- ptsrc_pb210 error

# Move Point Source Lead 210 data to ptsrc_pb210 and uncertainty to ptsrc_pb210 error of associated samples

# CHECK the following

Radioisotope data should have the following headings

 ### | File    | Pb-210   | Pb-210 error    | Bi-214  | Bi-214 error   | Pb-214    |Pb-214 error |  ptsrc_pb210    | ptsrc_pb210 error  | 


# Create a new .csv file from lab notebook for the sample weight data

Sample weights should have the following headings: 

### | Core    | Top of interval (cm)   | Center point of interval    |Base of interval (cm)  | sediment weight (g)    | 




Run cell below

In [ ]:
import pandas as pd
import numpy as np
import re

# Prompt user for file paths and output file name
csv1_path = input("Enter the path to the sample weight CSV file (e.g., /path/weights.csv): ")
csv2_path = input("Enter the path to the Canberra data CSV file (e.g., /path/canberra.csv): ")
output_file_name = input("Enter the path for the output CSV file (e.g., CORE_AgeModel_DATE.csv): ")


In [ ]:
#Load and merge data
# Load the CSV files
csv1 = pd.read_csv(csv1_path)
csv2 = pd.read_csv(csv2_path)

# Extract 'Center point of interval' from csv2 based on the median of the last digits in 'File'
csv2['Center point of interval'] = csv2['File'].apply(
    lambda x: np.median([int(num) for num in re.findall(r'\d+', x.split('_')[-1])])
)

# Merge CSV files on 'Center point of interval'
data = pd.merge(csv1, csv2, on='Center point of interval', how='left')

In [ ]:
# Prompt the user for the year of core
year_of_core = int(input("Enter the year of core (e.g., 2023): "))

In [ ]:
# Calculate activity and correction factors
data['Pb-210 activity Uncertainty (Bq-g)'] = data['Pb-210 error']/data['sediment weight (g)']
data['Pb-210 activity (Bq/g)'] = data['Pb-210'] / data['sediment weight (g)']
data['Pb-210 correction factor'] = data['ptsrc_pb210'] / 151031.56
data['Self absorb. Corrected Pb-210 activity (Bq/g)'] = (
    data['Pb-210 activity (Bq/g)'] / data['Pb-210 correction factor']
)
data['Bi-214 activity (Bq/g)'] = data['Bi-214'] / data['sediment weight (g)']
data['Pb-214 activity (Bq/g)'] = data['Pb-214'] / data['sediment weight (g)']

In [ ]:
# Calculate averaged supported activity of Bi-214 and Pb-214 (Bq/g)
data['Averaged supported activity of Bi-214 and Pb-214 (Bq/g)'] = (
    data['Bi-214 activity (Bq/g)'] + data['Pb-214 activity (Bq/g)']
) / 2

# Calculate background activity uncertainty (Bq/g)
data['Background activity uncertainty (Bq/g)'] = (
    (data['Bi-214 error'] + data['Pb-214 error']) / 2
) / data['sediment weight (g)']

In [ ]:
# Calculate Excess Pb-210 (Bq/g)
data['Excess Pb-210 (Bq/g)'] = (
    data['Self absorb. Corrected Pb-210 activity (Bq/g)'] -
    data['Averaged supported activity of Bi-214 and Pb-214 (Bq/g)']
)

# Determine surface activity (first interval value)
data['Surface activity'] = data['Excess Pb-210 (Bq/g)'].iloc[0]

# Calculate Age bp using the natural logarithm
data['Age bp'] = (1 / 0.03114) * np.log(data['Surface activity'] / data['Excess Pb-210 (Bq/g)'])


In [ ]:
# Calculate 'calendar years pre year of core'
data['calendar years pre year of core'] = year_of_core - data['Age bp']

# Save the final DataFrame to a CSV file
data.to_csv(output_file_name, index=False)

print(f"Calculations completed, data exported to '{output_file_name}'")

# Check the output data. Make sure data isnt *fishy*
Look at the column labeled 'Age'. Are the ages within the realm of possibility? If not, ask Asmara for help!

# Now plot it!
Run cell below 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd

# Ask for the age model file to be plotted (CSV format)
age_model_file = input("Enter the full path to the age model file to plot (e.g., /path/to/age_model.csv): ")
data = pd.read_csv(age_model_file)


In [ ]:
# Get the core name for the plot title
core_name = input("Enter the core name for the title: ")

# Ask for depths to label "calendar years pre year of core"
depths_to_label_input = input("Enter the depths (comma-separated) where 'calendar years pre year of core' should be labeled (or type 'all' to label all intervals): ")

if depths_to_label_input.lower() == 'all':
    depths_to_label = data['Center point of interval'].tolist()  # Label all intervals
else:
    depths_to_label = [float(depth.strip()) for depth in depths_to_label_input.split(",")]

In [ ]:
# Ask if any intervals have undetectable radioisotope amounts
missing_data_input = input("Are there any intervals with undetectable amounts of radioisotopes? (yes/no): ").strip().lower()

if missing_data_input == 'yes':
    missing_depths_input = input("Enter the depths (comma-separated) with undetectable radioisotopes: ")
    missing_depths = [float(depth.strip()) for depth in missing_depths_input.split(",")]
else:
    missing_depths = []


In [ ]:
# Define colors for the data series and error bars
excess_pb210_color = 'black'
excess_pb210_error_color = mcolors.to_rgba(excess_pb210_color, alpha=0.3)
supported_activity_color = 'grey'
supported_activity_error_color = mcolors.to_rgba(supported_activity_color, alpha=0.3)

# Ask for the folder to save the plot PDF and create a filename
save_location = input("Enter the full path where you want to save the plot PDF (e.g., /path/to/your/directory): ")
plot_filename = f"{core_name}_Age_Model.pdf"
save_path = save_location.rstrip('/') + "/" + plot_filename

In [ ]:
plt.figure(figsize=(3, 5))
plt.errorbar(
    data['Pb-210 activity (Bq/g)'], data['Center point of interval'], 
    xerr=data['Pb-210 activity Uncertainty (Bq-g)'], fmt='-', color=excess_pb210_color, 
    label='Pb-210 activity (Bq/unit)', capsize=5, linewidth=1, 
    ecolor=excess_pb210_error_color
)
plt.xscale('log')
plt.xlim(0.01, 10)
# Highlight intervals with missing radioisotopes using brown spans
for y in missing_depths:
    plt.axhspan(y - 0.5, y + 0.5, alpha=0.5, color='brown', 
                label='Undetectable radioisotope' if y == missing_depths[0] else None)

plt.title(f"{core_name} 210 Pb Uncorrected Activity", fontsize=18)
plt.xlabel("Bq/g", fontsize=14)
plt.ylabel("Depth (cm)", fontsize=14)
plt.gca().invert_yaxis()  # Show depth from surface (invert y-axis)
plt.grid(True, which='both', linestyle='-', linewidth=0.5, color='lightgray')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=1)
plt.tight_layout()
plt.savefig(save_path, format='pdf', bbox_inches='tight')
plt.show()


In [ ]:
import matplotlib.patches as patches

plt.figure(figsize=(5, 10))

# Calculate vertical errors from top/base to center of interval
depth_errors = np.array([
    [center - top, base - center]
    for top, center, base in zip(
        data['top of interval (cm)'],
        data['Center point of interval'],
        data['Base of interval (cm)']
    )
]).T  # shape (2, N)

# Calculate symmetric vertical errors
yerr = np.abs(data['Center point of interval'] - data['top of interval (cm)'])

# Calculate symmetric horizontal errors
xerr = data['Pb-210 activity Uncertainty (Bq-g)']

# Draw connecting line for Excess Pb-210 data points
plt.plot(
    data['Excess Pb-210 (Bq/g)'], data['Center point of interval'],
    color=excess_pb210_color, linewidth=1, zorder=1
)

# Draw error boxes for Excess Pb-210
for i in range(len(data)):
    x = data['Excess Pb-210 (Bq/g)'].iloc[i]
    y = data['Center point of interval'].iloc[i]
    width = xerr.iloc[i] * 2
    height = yerr.iloc[i] * 2
    rect = patches.Rectangle(
        (x - xerr.iloc[i], y - yerr.iloc[i]), width, height,
        linewidth=0.5, edgecolor='grey', facecolor='none'
    )
    plt.gca().add_patch(rect)

# Plot background activity with horizontal error bars only
plt.errorbar(
    data['Averaged supported activity of Bi-214 and Pb-214 (Bq/g)'],
    data['Center point of interval'],
    xerr=data['Background activity uncertainty (Bq/g)'],
    fmt='-', color=supported_activity_color, label='Background Activity',
    capsize=5, linewidth=1, ecolor=supported_activity_error_color
)

# Highlight missing intervals with brown spans
for y in missing_depths:
    plt.axhspan(y - 0.5, y + 0.5, alpha=0.5, color='brown',
                label='Undetectable radioisotope' if y == missing_depths[0] else None)

# Annotate the selected depths with rounded-up calendar years
for i, depth in enumerate(data['Center point of interval']):
    if depth in depths_to_label:
        year_value = data['calendar years pre year of core'].iloc[i]
        if not pd.isna(year_value):
            plt.text(
                data['Excess Pb-210 (Bq/g)'].iloc[i] + 0.05, depth,
                f'{int(np.ceil(year_value))}',
                fontsize=14, color='black', verticalalignment='center'
            )

plt.title(f"{core_name} Age Model", fontsize=18)
plt.xlabel("Bq/unit", fontsize=14)
plt.ylabel("Depth (cm)", fontsize=14)
plt.xscale('log')
plt.xlim(0.01, 1)
plt.gca().invert_yaxis()
plt.grid(True, which='both', linestyle='-', linewidth=0.5, color='lightgray')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=1)
plt.tight_layout()
plt.savefig(save_path, format='pdf', bbox_inches='tight')
plt.show()


# Well done!

#### When you've finished, go to Cell > All Output > Clear to be ready for the next user of this script.